--------------------------------------------------------------------------------
Travel time calculation using AnalyticalWell class, combined with 'omp removal'
--------------------------------------------------------------------------------

Basic example to calculate the travel time distribution per geological layer for a 
phreatic scheme, and to calculate the OMP removal. 

Import packages
-----------------

First we import the necessary python packages

In [1]:
import pandas as pd
from pathlib import Path
from set_cwd_to_project_root import project_root

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from pandas import read_csv
from pandas import read_excel
import math
from scipy.special import kn as besselk
import sutra2.Analytical_Well as AW
import sutra2.ModPath_Well as mpw
import sutra2.Transport_Removal as TR

# # get directory of this file
# path = Path(__file__).parent
# %load_ext autoreload
# %autoreload 2

ModuleNotFoundError: No module named 'set_cwd_to_project_root'

Step 1: Define the HydroChemicalSchematisation
-----------------------------------------------
The first step is to define the hydrogeochemistry of the system using the HydroChemicalSchematisation class.
In this class you specify the:

* Computational method ('analytical' or 'modpath').
* The schematisation type ('phreatic', 'semiconfined',...) 
note: schematisation types 'riverbankfiltration', 'basinfiltration' yet to be supported
* Input the relevant parameters for the porous media, the hydrochemistry, hydrology and the flow parameters of interest

The class parameters can be roughly grouped into the following categories:

* System.
* Settings.
* Porous Medium
* Hydrochemistry
* Hydrology
* Diffuse contamination
* Point Contamination
* Model size

Units of input are:

* Discharge : m3/d
* Time: days
* Length: meters
* Concentration: ug/L
* Temperature: degree C
* Depth: meters above sea level (m ASL)
* Density: kg/L
* DOC/TOC: mg/L

Lets start with a simple example defining a HydroChemicalSchematisation object for a phreatic aquifer:


In [ ]:
phreatic_schematisation = AW.HydroChemicalSchematisation(schematisation_type='phreatic',
                                                    computation_method='analytical',
                                                    well_discharge=-7500, #m3/day
                                                    recharge_rate=0.0008, #m/day
                                                    thickness_vadose_zone_at_boundary=5, #m
                                                    thickness_shallow_aquifer=10,  #m
                                                    thickness_target_aquifer=40, #m
                                                    hor_permeability_target_aquifer=35, #m/day
                                                    redox_vadose_zone='anoxic',
                                                    redox_shallow_aquifer='anoxic',
                                                    redox_target_aquifer='deeply_anoxic',
                                                    pH_target_aquifer=7.,
                                                    temp_water=11.,
                                                    diffuse_input_concentration = 100, #ug/L
                                                    )


The parameters from the HydroChemicalSchematisation class are added as attributes to
the class and can be accessed for example:

In [ ]:
phreatic_schematisation.schematisation_type
phreatic_schematisation.well_discharge
phreatic_schematisation.porosity_shallow_aquifer

0.35

If not defined, default values are used for the rest of the parameters. To view all parameters in the schematisation:

In [ ]:
phreatic_schematisation.__dict__

{'schematisation_type': 'phreatic',
 'computation_method': 'analytical',
 'removal_function': 'omp',
 'what_to_export': 'all',
 'well_name': 'well1',
 'temp_correction_Koc': True,
 'temp_correction_halflife': True,
 'biodegradation_sorbed_phase': True,
 'compute_thickness_vadose_zone': True,
 'ground_surface': 0.0,
 'thickness_vadose_zone_at_boundary': 5,
 'bottom_vadose_zone_at_boundary': -5.0,
 'thickness_shallow_aquifer': 10,
 'bottom_shallow_aquifer': -15.0,
 'thickness_target_aquifer': 40,
 'bottom_target_aquifer': -55.0,
 'thickness_full_capillary_fringe': 0.0,
 'porosity_vadose_zone': 0.35,
 'porosity_shallow_aquifer': 0.35,
 'porosity_target_aquifer': 0.35,
 'porosity_gravelpack': 0.35,
 'porosity_clayseal': 0.35,
 'moisture_content_vadose_zone': 0.2,
 'solid_density_vadose_zone': 2.65,
 'solid_density_shallow_aquifer': 2.65,
 'solid_density_target_aquifer': 2.65,
 'fraction_organic_carbon_vadose_zone': 0.001,
 'fraction_organic_carbon_shallow_aquifer': 0.0005,
 'fraction_organ

____________________________________


Step 2: Run the AnalyticalWell class
-------------------------------------
Next we create an AnalyticalWell object for the HydroChemicalSchematisation object we just made.


In [ ]:
phreatic_well = AW.AnalyticalWell(phreatic_schematisation)

Then we calculate the travel time for each of the zones unsaturated, shallow aquifer and target aquifer zones
by running the .phreatic() function for the well object. 

In [ ]:
phreatic_well.phreatic()

From the AnalyticalWell class two other important outputs are:

* df_particle - Pandas dataframe with data about the different flowlines per zone (unsaturated/shallow/target)
* df_flowline - Pandas dataframe with data about the flowlines per flowline (eg. total travel time per flowline)

____________________________________


Step 3: Collect removal parameters for the OMP (substance)
------------------------------------------------------------

You can retrieve the default removal parameters used to calculate the removal of organic micropollutants [OMP] 
in the SubstanceTransport class. The data are stored in a dictionary


In [ ]:
test_substance = TR.Substance(substance_name='benzene')
test_substance.substance_dict

NameError: name 'TR' is not defined